#### Disable Sleep Mode

In [1]:
import subprocess
import atexit
caffeinate_process = subprocess.Popen(["caffeinate", "-i"])
print("Caffeinate started to prevent sleep mode.")
def stop_caffeinate():
    caffeinate_process.terminate()
    print("Caffeinate stopped. Sleep mode enabled.")
atexit.register(stop_caffeinate)

Caffeinate started to prevent sleep mode.


<function __main__.stop_caffeinate()>

#### Import FMP URLs and FMP API Key

In [2]:
# import os
# try:
#     os.remove("stocks_data.db")
# except:
#     pass

%reload_ext autoreload 
%autoreload 2
import os
import pickle
import random
import sqlite3
import pandas as pd
from dotenv import load_dotenv
from ordered_set import OrderedSet
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None) 

from utils import db_funcs as db
from utils.df_funcs import *
from utils.fmp_funcs import *
from utils.graph_db_funcs import *
from utils.urls import url_stock_list, url_ccy_info, url_hist_price, url_mcap_24, url_mcap_19_24, url_mcap_14_19, url_all_forex_pairs, url_forex, url_company_info, url_company_info_short


load_dotenv()
API_KEY = os.getenv("API_KEY")


# import sqlite3
# db_file = "stocks_data.db"
# conn = sqlite3.connect(db_file)
# cursor = conn.cursor()
# conn.close()

neo4j+s://461e473a.databases.neo4j.io



#### Create new TICKERS list and CCY PAIRS list, and Pickle them

In [3]:
ccy_pairs  = get_forex_pairs(get_data(url_all_forex_pairs, API_KEY))
stock_records = get_stocks(get_data(url_stock_list, API_KEY))
all_symbols = get_symbols(stock_records)
random.shuffle(all_symbols)

# with open("files/ccy_pairs.pkl", "wb") as file:
#     pickle.dump(ccy_pairs, file)

# with open("files/all_symbols.pkl", "wb") as file:
#     pickle.dump(all_symbols, file)

#### Read the Pickle Files - TICKERS List and CCY PAIRS List

In [ ]:
with open('../files/ccy_pairs.pkl', 'rb') as file:
    ccy_pairs = pickle.load(file)


with open('../files/all_symbols.pkl', 'rb') as file:
    all_symbols = pickle.load(file)

all_symbols_set = OrderedSet(all_symbols)
print(len(all_symbols))

#### Write STOCK INFO Data to SQLITE

In [ ]:
with sqlite3.connect(db.db_file) as conn:
    cursor = conn.cursor()

    symbols_list = list(all_symbols_set)    
    company_data = []

    for i in range(0, len(symbols_list), 500):
        print(i)

        symbols = pd.read_sql_query("SELECT symbol FROM stock_info", conn)  
        existing_db_symbols = symbols.symbol.tolist()                  
        processed_symbols = OrderedSet(existing_db_symbols)

        non_db_symbols = [item for item in symbols_list if item not in processed_symbols]
        print(len(non_db_symbols))

        non_db_symbol_batch = list(non_db_symbols[0:500])
        non_db_symbol_batch_string = ",".join(non_db_symbol_batch)
        url = url_company_info_short.format(symbol=non_db_symbol_batch_string, api_key=API_KEY)

        company_data_batch = get_company_data(get_data(url, API_KEY))
        db.write_stock_data(company_data_batch, table_name="stock_info")

        print('processed_symbols', len(processed_symbols))
        time.sleep(5)
        print(time.time())

    print("Stock data successfully inserted/updated into SQLite database.")


#### Write FOREX Data to SQLITE

In [ ]:
with sqlite3.connect(db.db_file) as conn:
    cursor = conn.cursor()

    for idx, ccy_pair in enumerate(ccy_pairs):
        url = url_forex.format(forex_pair = ccy_pair, api_key = API_KEY)
        stock_data = get_close_data(get_data(url, API_KEY))

        print(url)
        print(idx, ccy_pair, stock_data[0:2])
        print("")

        try:
            db.write_ccy_data(cursor, ccy_pair, stock_data, 'exchange_rates')
            
        except Exception as e:
                print(f"Error fetching {ccy_pair}")
                print(f"{e}")

    print("Forex data successfully inserted/updated into SQLite database.")


#### Select a Subset of Stocks - For PRICES and MCAPS, Save the subset of TICKERS list as Pickle

In [ ]:
print("Run initiated")
conn = sqlite3.connect(db.db_file)

forex_rates = pd.read_sql_query("SELECT * FROM exchange_rates", conn).sort_values(by=['date'], ascending=False).reset_index(drop=True)   
stock_info = pd.read_sql_query("SELECT * FROM stock_info", conn)
print("Stock and Forex data loaded")


forex_rates['EUREUR'] = 1
stock_info, long_stock_info = transform_stock_info(stock_info, forex_rates, country_codes, stock_subset = 16000)
print("Stock and Forex data transformed")


all_symbols_16000 = stock_info.symbol.to_list()
with open("files/all_symbols_16000.pkl", "wb") as file:
    pickle.dump(all_symbols_16000, file)


with open('../files/all_symbols_16000.pkl', 'rb') as file:
    all_symbols_16000 = pickle.load(file)

all_symbols_16000_set = OrderedSet(all_symbols_16000)

#### Write PRICES to SQLITE

In [ ]:

with sqlite3.connect(db.db_file) as conn: 

    symbols_prices = pd.read_sql_query("SELECT distinct symbol from hist_prices", conn)
    existing_db_prices_symbols_set = OrderedSet(symbols_prices.symbol)
    processed_price_symbols_set = existing_db_prices_symbols_set.copy()
    print('Symbols in DB: ',len(processed_price_symbols_set))

    for idx, symbol in enumerate(all_symbols_16000_set):
        if symbol not in processed_price_symbols_set:
            url = url_hist_price.format(symbol=symbol, api_key=API_KEY)

            print(idx, symbol, url)

            try:
                stock_data = get_close_data(get_data(url, API_KEY))
                print(stock_data[0:1])
                db.insert_hist_prices(stock_data, conn)
                processed_price_symbols_set.add(symbol)
                print("")
                time.sleep(1)
            except Exception as e:
                print(f"Error fetching {symbol}")
                print(f"{e}")


#### Write MCAPS to Sqlite

In [ ]:

with sqlite3.connect(db.db_file) as conn:
    symbols_mcaps = pd.read_sql_query("SELECT distinct symbol from hist_mcaps", conn)
    existing_db_mcaps_symbols_set = OrderedSet(list(symbols_mcaps.symbol))
    processed_mcap_symbols = existing_db_mcaps_symbols_set.copy()

    for idx, symbol in enumerate(all_symbols_16000_set):
         if symbol not in processed_mcap_symbols:
             
            stock_data = []
            urls = [
                url_mcap_24.format(symbol=symbol, api_key=API_KEY),
                url_mcap_19_24.format(symbol=symbol, api_key=API_KEY),
                url_mcap_14_19.format(symbol=symbol, api_key=API_KEY),
            ]
            print(idx, symbol, urls)

            try:
                for url in urls:
                    element = get_mcap_data(get_data(url, API_KEY))
                    if element is not None:
                        stock_data.extend(element)

                db.insert_hist_mcaps(stock_data, conn)
                print(stock_data[0:2])
                processed_mcap_symbols.add(symbol)
                print("")
                time.sleep(0)

            except Exception as e:
                print(f"Error fetching {symbol}")
                print(f"{e}")
                time.sleep(10)
